# One-way route

## Imports

In [1]:
import googlemaps
import gmaps
import collections
from api_keys.api_key import api_key
collections.Iterable = collections.abc.Iterable
collections.Sequence = collections.abc.Sequence

import requests
import json

## Instaciate the API

In [2]:
key = api_key
client = googlemaps.Client(key)
gmaps.configure(api_key=key)

api_endpoint = "https://maps.googleapis.com/maps/api/place/textsearch/json"
base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

## Users input

In [3]:
# user input
start = input('Enter the start point: ')
destination = input('Enter the final destination: ')
max_dist = float(input('Enter the duration to drive per day, in hours: ')) * 80 * 1000

Enter the start point: madrid
Enter the final destination: paris
Enter the duration to drive per day, in hours: 3


## Defining the coordinates of the origin and the destination

### Origin

In [4]:
def origin_coord(start):
    start_id = client.places_autocomplete(start)[0]['place_id']

    start_coord = client.geocode(place_id=start_id)[0]['geometry']['location']

    start_list = (start_coord['lat'], start_coord['lng'])
    
    return start_list

In [5]:
origin_location = origin_coord(start)
origin_location

(40.4167754, -3.7037902)

### Destination

In [6]:
def destination_coord(destination):
    destination_id = client.places_autocomplete(destination)[0]['place_id']

    destination_coord = client.geocode(place_id=destination_id)[0]['geometry']['location']

    dest_list = (destination_coord['lat'], destination_coord['lng'])
    return dest_list

In [7]:
destination_location = destination_coord(destination)
destination_location

(48.856614, 2.3522219)

## Split the route based on the maximum distance to be traveled per day

In [14]:
def split_route(start, destination, max_dist, origin_location):
# Takes the origin and the destination and split the route into smaller pieces and get
# the coordinates of every point of the route.
    origin = client.places_autocomplete(start)[0]['description']
    destination = client.places_autocomplete(destination)[0]['description']

    route = client.directions(origin, destination, mode='driving')[0]['legs'][0]['steps']

    sum = 0
    coordinates = []
    for step in route:
        sum = sum + step['distance']['value']
        if sum >= max_dist:
            sum = 0
            coordinates.append(step['start_location'])
            
    coordinates.append(client.directions(origin, destination, mode='driving')[0]['legs'][0]['steps'][-1]['end_location'])
    
    stopover = []
    
    for coord in coordinates:
        coord_tuple = (coord['lat'], coord['lng'])
        stopover.append(coord_tuple)
        
    stopover.insert(0, origin_location)
    
    return stopover

In [15]:
route_coord = split_route(start, destination, max_dist, origin_location)
route_coord

[(40.4167754, -3.7037902),
 (41.5896386, -3.6990397),
 (43.3398577, -1.7522565),
 (45.7829853, -0.6677476),
 (48.7381702, 2.3192041),
 (48.85637149999999, 2.3532147)]

### Defining the waypoints coordinates

In [16]:
def waypoints(stop_coord):
    waypoints = stop_coord[1:-1]
    
    city_waypoint = []
    for coord in waypoints:
        city_location = client.places_nearby(coord, radius=20000, type='locality')['results'][0]['geometry']['location']
        city_coord = (city_location['lat'], city_location['lng'])
        city_waypoint.append(city_coord)
    
    return city_waypoint

In [17]:
waypoint_coord = waypoints(route_coord)
waypoint_coord

[(41.6692282, -3.6902166),
 (43.3381465, -1.7888495),
 (45.744175, -0.633389),
 (48.759255, 2.302553)]

## Finding nearest hotels and restaurant based in each stopover

In [18]:
def every_stop(waypoint_coord, destination_location, origin_location):
    lines_coord = waypoint_coord
    lines_coord.append(destination_location)
    lines_coord.insert(0, origin_location)
    
    return lines_coord

In [19]:
lines_coord = every_stop(waypoint_coord, destination_location, origin_location)
lines_coord

[(40.4167754, -3.7037902),
 (41.6692282, -3.6902166),
 (43.3381465, -1.7888495),
 (45.744175, -0.633389),
 (48.759255, 2.302553),
 (48.856614, 2.3522219)]

In [20]:
def location_marker(lines_coord,type_location):
    location_dest = []
    
    for stop in lines_coord:
        location_search = []
        locations = client.places_nearby(stop, radius=100000, type=type_location)['results']
        location_search.append(locations)

        location_names = []
        location_rating = []
        location_coord = []
        location_adress = []
        
        for result in location_search[0]:
            dictionary1 = {'name': result['name']}
            location_names.append(dictionary1)
            try:
                dictionary2 = {'rating': result['rating']}
                location_rating.append(dictionary2)
            except:
                dictionary2 = {'rating': "No Rating"}
                location_rating.append(dictionary2)
            
            tuple1 = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])
            dictionary3 = {'location': tuple1}
            location_coord.append(dictionary3)

            dictionary4 = {'adress': result['vicinity']}
            location_adress.append(dictionary4)

        location_info = []

        for i in range(len(location_names)):
            temp_dict = {}
            temp_dict.update(location_names[i])
            temp_dict.update(location_coord[i])
            temp_dict.update(location_rating[i])
            temp_dict.update(location_adress[i])
        
            location_info.append(temp_dict)
        
        location_dest.append(location_info)
        all_locations = []
        for i in range(len(location_dest)):
            all_locations = all_locations + location_dest[i]
    return all_locations

In [23]:
hotel_dest = location_marker(lines_coord, 'lodging')
hotel_dest

[{'name': 'Hotel Europa',
  'location': (40.4174939, -3.703499199999999),
  'rating': 4.3,
  'adress': 'Calle del Carmen, 4, Madrid'},
 {'name': 'Hotel Victoria 4',
  'location': (40.416379, -3.701742399999999),
  'rating': 4,
  'adress': 'Calle de la Victoria, 4, Madrid'},
 {'name': 'Petit Palace Posada del Peine',
  'location': (40.41571, -3.7060284),
  'rating': 4.4,
  'adress': 'Calle de Postas, 17, Madrid'},
 {'name': 'ME Madrid Reina Victoria',
  'location': (40.4145638, -3.7014472),
  'rating': 4.3,
  'adress': 'Calle del Príncipe, 14, Madrid'},
 {'name': 'Hotel Quatro Puerta Del Sol',
  'location': (40.4170661, -3.6999505),
  'rating': 3.9,
  'adress': 'Calle de Sevilla, 4, Madrid'},
 {'name': 'Hotel Regina Madrid',
  'location': (40.4179133, -3.699877499999999),
  'rating': 4.5,
  'adress': 'Calle de Alcalá, 19, Madrid'},
 {'name': 'INNSiDE by Meliá Madrid Gran Vía',
  'location': (40.4205878, -3.7040355),
  'rating': 4.2,
  'adress': 'Calle de Mesonero Romanos, 13, Madrid'},


In [24]:
rest_dest = location_marker(lines_coord, 'restaurant')
rest_dest

[{'name': 'Restaurante Krüger',
  'location': (40.4246643, -3.7133071),
  'rating': 3.9,
  'adress': 'Calle de la Princesa, 5, Madrid'},
 {'name': 'La Chulapa de Alcalá',
  'location': (40.41852529999999, -3.6980617),
  'rating': 4,
  'adress': 'Calle de Alcalá, 35, Madrid'},
 {'name': 'Corral de la Morería',
  'location': (40.4126689, -3.7142336),
  'rating': 4.5,
  'adress': 'Calle de la Moreria, 17, Madrid'},
 {'name': 'Taberna del Alabardero de Madrid',
  'location': (40.4186021, -3.710921599999999),
  'rating': 4.3,
  'adress': 'Calle de Felipe V, 6, Madrid'},
 {'name': 'Casa de Asturias',
  'location': (40.4081497, -3.699811),
  'rating': 3.8,
  'adress': 'Calle de Argumosa, 4, Madrid'},
 {'name': 'Hotel Intur Palacio San Martín',
  'location': (40.4186427, -3.7067589),
  'rating': 4.4,
  'adress': 'Plaza de San Martín, 5, Madrid'},
 {'name': 'VP Jardín de Recoletos',
  'location': (40.4223075, -3.689168100000001),
  'rating': 4.5,
  'adress': 'Calle de Gil de Santivañes, 6, Madr

## Getting the famous tourist attractions

In [25]:
query = "natural feature"

In [26]:
def attraction_feature(place, query):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    querystring = {"query": f'{query} in' + place, "key": key}
    response = requests.request("GET", url, params=querystring)
    data = json.loads(response.text)

    locations = []

    if data['status'] == 'OK':
        results = data['results']

        attraction_names = []
        attraction_rating = []
        attraction_coord = []

        for result in results:
            dictionary1 = {'name': result['name']}
            attraction_names.append(dictionary1)

            try:
                dictionary2 = {'rating': result['rating']}
                attraction_rating.append(dictionary2)
            except:
                dictionary2 = {'rating': "No Rating"}
                attraction_rating.append(dictionary2)

            tuple1 = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])
            dictionary3 = {'location': tuple1}
            attraction_coord.append(dictionary3)

        attraction_info = []

        for i in range(len(attraction_names)):
            temp_dict = {}
            temp_dict.update(attraction_names[i])
            temp_dict.update(attraction_coord[i])
            temp_dict.update(attraction_rating[i])
            attraction_info.append(temp_dict)


#     else:
#             print("Error:", data['status'])

    return attraction_info

In [27]:
natural_dest = attraction_feature(start, query)
natural_dest

[{'name': 'Berlin Park',
  'location': (40.4504072, -3.675755799999999),
  'rating': 4.4},
 {'name': 'Casa de Campo Lake',
  'location': (40.4187716, -3.7326261),
  'rating': 4.6},
 {'name': 'Green Pond (waterfall)',
  'location': (40.7641654, -3.9051619),
  'rating': 4.7},
 {'name': "El Pardo's Dam",
  'location': (40.5387583, -3.7886219),
  'rating': 4.2},
 {'name': 'Manzanares River Linear Park',
  'location': (40.3775309, -3.6865892),
  'rating': 4.6},
 {'name': 'Bosque del Recuerdo',
  'location': (40.4120836, -3.6871405),
  'rating': 4.5},
 {'name': "Park plantation of the Berro's fountain",
  'location': (40.4215611, -3.660819699999999),
  'rating': 4.7},
 {'name': "Madrid's Oldest Tree",
  'location': (40.4156301, -3.688006199999999),
  'rating': 4.7},
 {'name': 'Parque Juan Carlos I',
  'location': (40.460916, -3.6047052),
  'rating': 4.6},
 {'name': 'La Montaña Park',
  'location': (40.4229964, -3.716896),
  'rating': 4.8},
 {'name': 'Cascada del Purgatorio',
  'location': (4

## Generate the map, with the main route and the stopover, including hotels and restaurants for each stop.

In [32]:
def main_route(origin_coord, destination_coord, waypoints, hotel_markers, rest_markers, natural_markers, lines_coord):
    fig = gmaps.figure()
    
    main = gmaps.directions_layer(origin_coord, destination_coord, waypoints=waypoints)
    
    for coord in lines_coord:
        
    
    hotel_location = [hotel['location'] for hotel in hotel_markers]
    info_box_template = """
    <dl>
    <dt>Name</dt><dd>{name}</dd>
    <dt>Rating</dt><dd>{rating}</dd>
    <dt>Adress</dt><dd>{adress}</dd>
    </dl>
    """
    info_hotel = [info_box_template.format(**hotel) for hotel in hotel_markers]
    hotel_markers = gmaps.symbol_layer(hotel_location, hover_text='Hotel', fill_color='red', stroke_color='yellow',  scale=4, info_box_content=info_hotel)
    
    rest_location = [rest['location'] for rest in rest_markers]
    info_box_template = """
    <dl>
    <dt>Name</dt><dd>{name}</dd>
    <dt>Rating</dt><dd>{rating}</dd>
    <dt>Adress</dt><dd>{adress}</dd>
    </dl>
    """
    info_rest = [info_box_template.format(**rest) for rest in rest_markers]
    rest_markers = gmaps.symbol_layer(rest_location, hover_text='Restaurant', fill_color='red', stroke_color='blue',  scale=4, info_box_content=info_rest)
    
    natural_location = [natural['location'] for natural in natural_markers]
    info_box_template = """
    <dl>
    <dt>Name</dt><dd>{name}</dd>
    <dt>Rating</dt><dd>{rating}</dd>
    </dl>
    """
    info_natural = [info_box_template.format(**natural) for natural in natural_markers]
    natural_markers = gmaps.symbol_layer(natural_location, hover_text='Natural Feature', fill_color='red', stroke_color='green',  scale=4, info_box_content=info_natural)
    
    fig.add_layer(main)
    fig.add_layer(hotel_markers)
    fig.add_layer(rest_markers)
    fig.add_layer(natural_markers)
    
    return fig

In [34]:
main_route(origin_location, destination_location, waypoint_coord, hotel_dest, rest_dest, natural_dest, lines_coord)

Figure(layout=FigureLayout(height='420px'))